In [2]:
import os
print(os.environ['PATH'])

import tensorflow as tf
from tensorflow import keras

from nn.models import C6_4l_clf_mini, swish_activation
from nn import datasets
from hstar import gghzz, c6, msq
from hzz import zpair, angles

import numpy as np
import matplotlib.pyplot as plt

/u/griesemx/ggZZ_work/venv/bin:/raven/u/griesemx/ggZZ_work/venv/bin:/raven/u/griesemx/.vscode-server-insiders/cli/servers/Insiders-8b7eb51f54d7e1492d9baf70454ab6547a4ff9df/server/bin/remote-cli:/mpcdf/soft/SLE_15/packages/x86_64/Modules/5.4.0/bin:/u/griesemx/bin:/usr/local/bin:/usr/bin:/bin:/usr/lib/mit/bin:/usr/lib/mit/sbin:/afs/ipp/amd64_sles15/bin:/mpcdf/soft/SLE_15/packages/x86_64/find-module/1.0/bin:/afs/ipp/amd64_sles15/bin


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
GEN=2
SEED=373485
OUTPUT_DIR='outputs/single'
SAMPLE_DIR='..'

In [ ]:
sample = gghzz.Process(  
    (1.4783394, SAMPLE_DIR + '/ggZZ2e2m_all_new.csv', 1e6),
    (0.47412769, SAMPLE_DIR + '/ggZZ4e_all_new.csv', 1e6),
    (0.47412769, SAMPLE_DIR + '/ggZZ4m_all_new.csv', 1e6)
)

In [ ]:
sample.events = sample.events.shuffle(SEED).iloc[4000000:]


z_chooser = zpair.ZPairChooser(bounds1=(50,115), bounds2=(50,115), algorithm='leastsquare')
l1_1, l2_1, l1_2, l2_2 = sample.events.filter(z_chooser)

kin_variables = angles.calculate(l1_1, l2_1, l1_2, l2_2)

c6_values = [20]

c6_mod = c6.Modifier(amplitude_component = msq.Component.SBI, c6_values = [-5,-1,0,1,5])
c6_weights, c6_prob = c6_mod.modify(sample=sample, c6=c6_values)

In [ ]:
test_data = datasets.build_dataset_tf(x_arr = kin_variables,
                                      param_values = c6_values,
                                      signal_weights = c6_weights,
                                      background_weights = tf.convert_to_tensor(sample.events.weights), 
                                      normalization = 1)

test_data = tf.random.shuffle(test_data, seed=SEED)

In [ ]:
print(test_data, test_data.shape)

In [ ]:
model = keras.models.load_model(OUTPUT_DIR + '/ckpt/checkpoint.model_1_m4l.keras', custom_objects={'C6_4l_clf_mini': C6_4l_clf_mini, 'swish_activation': swish_activation})

In [ ]:
predictions = []
real_ratios = []

arr_len = test_data.shape[0]/len(c6_values)

for i in range(len(c6_values)):
    c6 = c6_values[i]
    data = test_data[int(i*arr_len):int((i+0.5)*arr_len),0][:,np.newaxis]
    pred = model.predict(data)
    predictions.append(pred)

    real_ratio = test_data[int(i*arr_len):int((i+0.5)*arr_len),2]/test_data[int((i+0.5)*arr_len):int((i+1)*arr_len),2]
    real_ratios.append(real_ratio)

predictions = tf.convert_to_tensor(predictions)

ratios = tf.squeeze(predictions/(1-predictions), axis=2)

print(ratios)

real_ratios = tf.convert_to_tensor(real_ratios)

print(real_ratios)

In [ ]:
real = real_ratios.numpy()
pred = ratios.numpy()

lnspc = np.linspace(0.6,1.2)

plt.figure(figsize=(6,6))

plt.plot(lnspc, lnspc, color='dimgray', linestyle=(0,(5,10)), label='NN estimate = true')
plt.scatter(real, pred, s=10, marker='x', label='Estimated data')
plt.xlabel(u'True ratio   $P(x|c_6)/P_0(x)$')
plt.ylabel(u'NN estimated   $P(x|c_6)/P_0(x)$')
plt.xlim(0.0,2.0)
plt.ylim(0.0,2.0)
plt.legend()

In [ ]:
hist_prm = ''

with open(f'outputs/m4l/single/history/history_2_m4l.txt', 'r') as hist_file:
    hist_prm = hist_file.readlines()

hist_prm = [ np.array(el.replace('[','').replace(']','').replace(' ','').split(','), dtype=float) for el in hist_prm[0].split('][')]

t_loss_prm = np.array(hist_prm[0])
v_loss_prm = np.array(hist_prm[1])

epochs = range(1,101)

In [ ]:
fig = plt.figure(figsize=(8,6))

#ax1.set_xticklabels([])

plt.plot(epochs, t_loss_prm, 'b', label='Training loss')
plt.xlabel('epochs []')
plt.ylabel('loss []')
#ax2.legend()

plt.plot(epochs, v_loss_prm, 'r', label='Validation loss')
#ax1.set_ylabel('loss []')
plt.legend()

fig.tight_layout()

plt.show()